In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
path = {"executable_path":"/usr/local/bin/chromedriver"}
browser = Browser("chrome", **path, headless=True)

# Austin Brewery Data Collection & Cleaning

In [3]:
url = 'https://bigworldsmallgirl.com/central-texas-brewery-guide/#:~:text=With%20over%2058%20breweries%20in,love%20about%20our%20craft%20beer.'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
brewery_data = soup.find('div', class_='entry-content clearfix')
brewery_list = brewery_data.find_all('p')

In [24]:
breweries = []
addresses = []
for brewery in brewery_list:
    if (brewery.a and brewery.a.strong):
        brewery_name = brewery.a.strong.text
        brewery_address = brewery.a.next_sibling
        breweries.append(brewery_name)
        addresses.append(brewery_address)
    elif (brewery.strong and brewery.strong.a):
        brewery_name = brewery.strong.a.text
        brewery_address = brewery.strong.next_sibling
        breweries.append(brewery_name)
        addresses.append(brewery_address)
addresses[22] = '417 Red River Street, 78701'
addresses[34] = '3220 Manor Road, 78723'

In [30]:
breweries_df = pd.DataFrame({'Brewery': breweries, 'Address': addresses})
breweries_df['Street_Address'] = breweries_df['Address'].str.extract('(\d{3,5}\s\w[^),]+)', expand=True)
breweries_df['Zipcode'] = breweries_df['Address'].str.extract('(\s7\d\d\d\d)', expand=True)
breweries_df['Address'] = breweries_df['Address'].str.lstrip()
breweries_df.loc[(breweries_df['Brewery'] == 'Orf Brewing'), 'Street_Address']= '4700-F Burleson Road'
breweries_df=breweries_df.drop([30,36,58,59])
breweries_df['Zipcode'][16] = 78641
breweries_df['Zipcode'] = breweries_df['Zipcode'].astype(int)
breweries_df = breweries_df.reset_index(drop=True)
area = []
area += 12 * ['North']
area += 10 * ['Northern Suburbs']
area += 2 * ['Central/Downtown']
area += 9 * ['South']
area += 11 * ['East']
area += 12 * ['West']
breweries_df['Austin_Area']=area
breweries_df_clean = breweries_df[['Brewery', 'Street_Address', 'Zipcode', 'Austin_Area']]
breweries_df_clean.head(10)

,Brewery,Street_Address,Zipcode,Austin_Area
0,4th Tap Brewing Cooperative,10615 Metric Boulevard,78758,North
1,Adelbert’s Brewery,2314 Rutland Drive,78758,North
2,Austin Beerworks,3001 Industrial Terrace,78758,North
3,Black Star Co-Op,7020 Easy Wind Drive,78752,North
4,Brewtorium,6015 Dillard Circle,78752,North
5,Celis Brewing,10001 Metric Boulevard,78758,North
6,Circle Brewing,2340 W Braker Lane,78758,North
7,Draught House,4112 Medical Parkway,78756,North
8,Hopsquad Brewing,2307 Kramer Lane,78758,North
9,Flying Man Brewing,2400 Patterson Industrial Drive,78660,North


In [31]:
breweries_df_clean.to_csv('data/brewery_list.csv')

# Top Rated Beers From Austin Breweries Data

In [36]:
untappd_urls = pd.read_csv('data/untappd_urls.csv')
url_list = untappd_urls['Untappd'].to_list()

In [37]:
brewery_id = []
x=0
for brewery in breweries_df_clean['Brewery']:
    brewery_id += 5 * [x]
    x +=1
brewery_id.pop(263)

52

In [39]:
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]

for url in url_list:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    beer = soup.find('div', class_='beer-container')
    
    beer_name_search = beer.find_all('p', class_='name')[:5]
    
    beer_type_search = beer.find_all('p', class_='style')[:5]

    beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
    beer_abv_search2 = beer.find_all('p', class_='abv')[:5]

    beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
    beer_ibu_search2 = beer.find_all('p', class_='ibu')[:5]

    beer_rating_search = beer.find_all('span', class_='num')[:5]

    beer_popular = beer.find_all('div', class_='details-item raters')[:5]
    beer_popular2 = beer.find_all('p', class_='raters')[:5]
    
    for paragraph in beer_name_search:
        if paragraph.a:
            beer_name.append(paragraph.a.text)

    for paragraph in beer_type_search:
        if paragraph.text:
            beer_type.append(paragraph.text)

    for paragraph in beer_abv_search:
        if paragraph.text:
            beer_abv.append(paragraph.text)
    for paragraph in beer_abv_search2:
        if paragraph.text:
            beer_abv.append(paragraph.text)

    for paragraph in beer_ibu_search:
        if paragraph.text:
            beer_ibu.append(paragraph.text)
    for paragraph in beer_ibu_search2:
        if paragraph.text:
            beer_ibu.append(paragraph.text)

    for paragraph in beer_rating_search:
        if paragraph.text:
            beer_rating.append(paragraph.text)

    for paragraph in beer_popular:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)
    for paragraph in beer_popular2:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)

In [40]:
beer_df = pd.DataFrame({'Brewery_ID':brewery_id,
                       'Name':beer_name,
                       'Type': beer_type,
                       'ABV': beer_abv,
                       'IBU':beer_ibu,
                       'Rating':beer_rating,
                       'Rating_Count':beer_number_rating})
beer_df.head(10)

,Brewery_ID,Name,Type,ABV,IBU,Rating,Rating_Count
0,0,Long Walk,IPA - American,\n7% ABV,\n57 IBU,(3.61),"\n5,723 Ratings"
1,0,Kung Fu Robot,IPA - American,\n7% ABV,\n43 IBU,(3.65),"\n4,714 Ratings"
2,0,Vitality Light Ale,Blonde Ale,\n5% ABV,\n17 IBU,(3.46),"\n2,687 Ratings"
3,0,Sun Eater,Gruit / Ancient Herbed Ale,\n4.8% ABV,\nN/A IBU,(3.48),"\n2,579 Ratings"
4,0,Highland Scoundrel,Scotch Ale / Wee Heavy,\n10.6% ABV,\n20 IBU,(3.84),"\n2,404 Ratings"
5,1,Naked Nun,Wheat Beer - Witbier,\n5.8% ABV,\nN/A IBU,(3.6),"\n10,933 Ratings"
6,1,Tripel B,Belgian Tripel,\n9.3% ABV,\nN/A IBU,(3.87),"\n10,253 Ratings"
7,1,Philosophizer,Farmhouse Ale - Saison,\n7.8% ABV,\n36 IBU,(3.8),"\n9,746 Ratings"
8,1,Hibiscus Saison,Farmhouse Ale - Saison,\n5.6% ABV,\n33 IBU,(3.65),"\n7,305 Ratings"
9,1,Flyin' Monks,Belgian Quadrupel,\n10.9% ABV,\nN/A IBU,(3.95),"\n6,308 Ratings"


In [41]:
beer_df['ABV']=beer_df['ABV'].str.replace('% ABV','')
beer_df['ABV']=beer_df['ABV'].str.replace('\n','')

beer_df['ABV']=pd.to_numeric(beer_df['ABV'],errors='coerce')

beer_df['IBU']=beer_df['IBU'].str.replace(' IBU','')
beer_df['IBU']=beer_df['IBU'].str.replace('\n','')
beer_df['IBU']=pd.to_numeric(beer_df['IBU'],errors='coerce')

beer_df['Rating']=beer_df['Rating'].str.replace('(','')
beer_df['Rating']=beer_df['Rating'].str.replace(')','')
beer_df['Rating']=pd.to_numeric(beer_df['Rating'],errors='coerce')

beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace('\n','')
beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace(',','')
beer_df['Rating_Count']=beer_df['Rating_Count'].str.replace(' Ratings','')
beer_df['Rating_Count']=pd.to_numeric(beer_df['Rating_Count'],errors='coerce')

beer_df_clean = beer_df
beer_df_clean.head(10)

,Brewery_ID,Name,Type,ABV,IBU,Rating,Rating_Count
0,0,Long Walk,IPA - American,7.0,57.0,3.61,5723
1,0,Kung Fu Robot,IPA - American,7.0,43.0,3.65,4714
2,0,Vitality Light Ale,Blonde Ale,5.0,17.0,3.46,2687
3,0,Sun Eater,Gruit / Ancient Herbed Ale,4.8,NaN,3.48,2579
4,0,Highland Scoundrel,Scotch Ale / Wee Heavy,10.6,20.0,3.84,2404
5,1,Naked Nun,Wheat Beer - Witbier,5.8,NaN,3.60,10933
6,1,Tripel B,Belgian Tripel,9.3,NaN,3.87,10253
7,1,Philosophizer,Farmhouse Ale - Saison,7.8,36.0,3.80,9746
8,1,Hibiscus Saison,Farmhouse Ale - Saison,5.6,33.0,3.65,7305
9,1,Flyin' Monks,Belgian Quadrupel,10.9,NaN,3.95,6308


In [42]:
beer_df_clean.to_csv('data/beer_list.csv')